In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

### 0) Prepare data

In [2]:
bc = datasets.load_breast_cancer()

In [3]:
X,y = bc.data,bc.target

In [6]:
X.shape

(569, 30)

In [4]:
n_samples,n_features = X.shape

In [7]:
print(n_samples,n_features)

569 30


In [8]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1234)

### Scale

In [9]:
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [12]:
X_train[0]

array([-0.36180827, -0.26521011, -0.31715702, -0.46713841,  1.80382609,
        1.18174184, -0.51689239,  0.10653677, -0.39005152,  1.39140136,
        0.14370971, -0.12075458,  0.16013008, -0.13255126, -0.58631633,
       -0.12476339, -0.57865272,  0.10906824, -0.28187354, -0.1889237 ,
       -0.25710149, -0.24033176, -0.2441722 , -0.36688232,  0.5448709 ,
        0.24812607, -0.71088618, -0.07967528, -0.52798733,  0.2506337 ])

In [13]:
X_test[0]

array([-0.70853151,  0.15314749, -0.70166016, -0.68807808,  0.36458732,
       -0.18758051, -0.59503023, -0.70729882,  0.16359564,  0.25801545,
       -0.72412663, -0.63626737, -0.73863481, -0.58065458, -0.59345259,
       -0.45593318, -0.44087965, -1.02140464, -0.7489644 , -0.42667137,
       -0.61187447,  0.47306762, -0.61265016, -0.58724931,  0.94933428,
        0.24998499, -0.00814052, -0.51072015,  0.50433894,  0.28112023])

In [14]:
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

Row to column vector

In [15]:
y_train = y_train.view(y_train.shape[0],1)
y_test = y_test.view(y_test.shape[0],1)

### 1.Model

In [16]:
# f = wx + b , sigmoid at end

class LogisticRegression(nn.Module):
    
    def __init__(self,n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features,1)
        
    def forward(self,x):
        y_pred = torch.sigmoid(self.linear(x))
        return(y_pred)
    
    
model = LogisticRegression(n_features)

### Loss and Optimiser

In [17]:
learning_rate = 0.01

#Binary Cross Entropy
criterion = nn.BCELoss()

optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

### 3.Training loop

In [19]:
num_epochs = 100

for epoch in range(num_epochs):
    
    #forward pass and loss
    y_pred = model(X_train)
    loss = criterion(y_pred,y_train)
    
    #backward pass
    loss.backward()
    
    #update
    optimizer.step()
    
    optimizer.zero_grad()
    
    if (epoch+1) % 10 == 0:
        print(f'epoch : {epoch+1},loss : {loss.item():.4f}')

epoch : 10,loss : 0.5409
epoch : 20,loss : 0.4503
epoch : 30,loss : 0.3914
epoch : 40,loss : 0.3499
epoch : 50,loss : 0.3190
epoch : 60,loss : 0.2949
epoch : 70,loss : 0.2755
epoch : 80,loss : 0.2596
epoch : 90,loss : 0.2462
epoch : 100,loss : 0.2346


In [20]:
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(acc)

tensor(0.8947)
